### Import packages

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
from urllib.request import urlopen
from tqdm import tqdm
import random
import time
from selenium import webdriver
from langdetect import detect

### Global variables

In [2]:
project_features = [
    'pro_link',
    'title',
    'status',
    'percentage_fund',
    'funded',
    'target',
    'end_date',
    'backers',
    'subcategory',
    'thumbnail_type',
    'number_rewards',
    'min_price',
    'max_price',
    'num_news',
    'num_comments',
    'num_contributions',
    'creater_link',
    'language',
    'short_des'
]

reward_features = [
    'pro_link',
    'price',
    'num_backers',
    'has_media'
]

creater_features = [
    'creater_link',
    'name',
    'num_created',
    'num_backed',
    'num_followed'
]

timeline_features = [
    'pro_link',
    'date',
    'amount'
] 

comment_features = [
    'pro_link',
    'comment_txt'
]

### Functions

In [3]:
def create_url(page):
    return 'https://www.kisskissbankbank.com/en/discover?project[successful]=on&filter=all&page=' + str(page)

In [4]:
def extract_creater(creater_link):
    # 1. Go to the page of the creater
    # 1.1 Load the website by driver
    chrome_path = './chromedriver.exe'
    driver = webdriver.Chrome(chrome_path)
    driver.get(creater_link)
    time.sleep(5)
    
    # 1.2 Accept the cookie
    cookie_question = driver.find_element_by_xpath('//*[@id="Header-react-component"]/div[1]/div/div/div/div/button[2]')
    cookie_question.click()
    
    # 1.3 Load the html page
    html = driver.page_source
    bs = BeautifulSoup(html, 'html.parser')
    
    # 2. Get information about the creater
    name = bs.find('h1', 'k-Title k-Title--quinary').text
    span = bs.findAll('span', 'k-Badge k-HorizontalNav__badge')
    num_created = int(span[0].text)
    num_backed = int(span[1].text)
    num_followed = int(int(span[2].text))
    
    # 3. Quit the driver
    driver.quit()
    
    return name, num_created, num_backed, num_followed

In [44]:
def extract_data(url):
    # 1. Go to the page of the project
    # 1.1 Load the website by driver
    chrome_path = './chromedriver.exe'
    driver = webdriver.Chrome(chrome_path)
    driver.get(url)
    time.sleep(5)
    
    # 1.2 Accept the cookie
    cookie_question = driver.find_element_by_xpath("//*[@id=\"App-react-component\"]/div[1]/div/div/div/div/button[2]")
    cookie_question.click()
    
    # 1.3 Load the html page
    html = driver.page_source
    bs = BeautifulSoup(html, 'html.parser')
    
    # 2. Get information about the project
    # Get pro_link
    pro_link = url
    
    # Get title
    title = bs.find('h1', 'title__StyledTitle-sc-46lshq-0 kxkLAV titles__StyledTitle-sc-1v04wsx-0 gwAQhx k-u-align-center').contents[0]
    
    # Get status
    status = bs.find('span', 'text__StyledText-sc-1jqe2sw-0 kKHxol k-u-color-font1 k-u-weight-regular project-state__StyledText-w82mew-2 dClAPF').contents[0]
    
    #Get percentage_fund
    percentage_fund = bs.find('div', 'text__StyledText-sc-1jqe2sw-0 kKHxol k-u-color-font1 k-u-line-height-normal k-u-weight-regular progress__StyledText-ycznm1-1 iXebEz').contents[0]
    
    # Get funded, target, end_date, backers
    info = bs.findAll('div', 'text__StyledText-sc-1jqe2sw-0 kKHxol k-u-color-font1 k-u-weight-bold info__StyledBigText-lcf1ol-0 clfvNf')
    funded = info[2].contents[0] 
    target = bs.findAll('div', 'text__StyledText-sc-1jqe2sw-0 kKHxol k-u-color-font1 k-u-weight-light info__StyledSmallText-lcf1ol-1 gDdVEG')[2].contents[0]
    end_date = info[1].contents[0]
    backers = info[0].contents[0]
    
    # Get subcategory
    subcategories = bs.findAll('span','text__StyledText-sc-1jqe2sw-0 kKHxol k-u-color-font1 k-u-size-micro k-u-weight-regular')
    temp_sub = []
    for s in subcategories:
        cate = ''
        for k in range (0,len(s.contents),2):
            if(k == 0):
                cate = cate + s.contents[k]
            else:
                cate = cate + ' ' + s.contents[k]
        temp_sub.append(cate)
    subcategory = ','.join(temp_sub)
    
    # Get thumbnail_type
    thumbnail_type = bs.find('img','project-media__StyledAvatar-bus9q7-1 oMMta').get('src').split('.')[-1]
    
    # Get number_rewards, min_price, max_price, and 3. Get information about rewards
    rewards = bs.findAll('div','marger__StyledMarger-sc-1qqifp5-0 kXGmDY')
    price = []
    num_backers = []
    reward = [] 
    for j in range(len(rewards)):
        rj = rewards[j]
            
        # Get remaining variables
        price_j = rj.findAll('h2', 'title__StyledTitle-sc-46lshq-0 edmigd k-RewardCard__title k-u-margin-bottom-double k-u-margin-top-none')
        if(len(price_j) == 0):
            break
        try:
            price_j = float(price_j[0].contents[0].replace('€','').replace(',',''))
        except:
            break
        price.append(price_j)
        backers_j = rj.findAll('span', 'text__StyledText-sc-1jqe2sw-0 kKHxol k-u-color-font1 k-u-size-micro k-u-weight-regular k-RewardCard__infos k-RewardCard__infos--hasBottomMargin k-RewardCard__infos--disabled')
        bj = 0
        if(len(backers_j) != 0 and backers_j[0].contents[0].replace(' ','').isnumeric()):
            bj = int(backers_j[0].contents[0])
        num_backers.append(bj)
        
        # Get has_media
        img = rj.findAll('img','k-RewardCard__image k-RewardCard__image--disabled')
        if(len(img) > 0):
            has_media_j = True
        else:
            has_media_j = False
            
        reward.append([pro_link,price_j,bj,has_media_j])
    if(len(price) == 0):
        min_price = -1
        max_price = -1
    else:
        min_price = np.min(np.array(price))
        max_price = np.max(np.array(price))
    number_rewards = len(price)
    
    # Get num_news, num_comments, num_contributions
    information = bs.findAll('a', 'k-NavBar__link')
    news_bs = information[2].findAll('span', 'badge__StyledBadge-sc-7liuod-0 eucluz k-Badge k-Badge--spaced')
    comments_bs = information[3].findAll('span', 'badge__StyledBadge-sc-7liuod-0 eucluz k-Badge k-Badge--spaced')
    contributions_bs = information[4].findAll('span', 'badge__StyledBadge-sc-7liuod-0 eucluz k-Badge k-Badge--spaced')
    
    if(len(news_bs) == 0):
        num_news = 0
    else:
        num_news = int(news_bs[0].contents[0])

    if(len(comments_bs) == 0):
        num_comments = 0
    else:
        num_comments = int(comments_bs[0].contents[0])

    if(len(contributions_bs) == 0):
        num_contributions = 0
    else:
        num_contributions = int(contributions_bs[0].contents[0])
        
    # Get creater_link
    creater_link = 'https://www.kisskissbankbank.com' + bs.find('a','owner-info__StyledOwnerGrid-tqxc8c-0 jnrkmW').get('href')
    
    # Get language
    short_des = bs.findAll('p',{'data-test-id':'short-description'})[0].contents[0]
    language = detect(short_des)
    
    # Get short description
    short_des = bs.findAll('p',{"data-test-id":"short-description"})[0].contents[0]
    
    project = [
        pro_link,
        title,
        status,
        percentage_fund,
        funded,
        target,
        end_date,
        backers,
        subcategory,
        thumbnail_type,
        number_rewards,
        min_price,
        max_price,
        num_news,
        num_comments,
        num_contributions,
        creater_link,
        language,
        short_des
    ]
    
    
    # 5. Get information about timeline
    '''
    timeline = []
    driver.find_element_by_xpath("//a[contains(., 'Contributions')]").click();
    time.sleep(2)
    html = driver.page_source
    bs = BeautifulSoup(html, 'html.parser')
    
    date, amount = [], []
    while True:
        try:
            driver.find_element_by_xpath("//button[contains(., 'Load more')]").click();
            time.sleep(5)
        except:
            time.sleep(5)
            break        
    html = driver.page_source
    bs = BeautifulSoup(html, 'html.parser')
    contributions = bs.findAll('div', 'backer-card__StyledCard-sc-1buoqas-0 TgtUZ')
    print(len(contributions))
    for contribution in contributions:
        spans = contribution.find('p','text__StyledText-sc-1jqe2sw-0 kKHxol k-u-color-font1 k-u-line-height-normal k-u-size-micro k-u-weight-light k-u-margin-none').findAll('span')
        result = [span.text.strip() for span in spans]
        if len(result) > 1: 
            #date.append(result[1])
            #amount.append(result[0].replace('€', ''))
            timeline.append([pro_link, result[1], result[0].replace('€', '')])


    # 6. Get information about comments
    comment = []
    driver.find_element_by_xpath("//a[contains(., 'Comments')]").click();
    time.sleep(5)
    html = driver.page_source
    bs = BeautifulSoup(html, 'html.parser')
    
    comments_txt = []
    #replies_txt = []
    
    while True:
        try:
            driver.find_element_by_xpath("//button[contains(., 'Load more')]").click();
            time.sleep(5)
        except:
            time.sleep(5)
            break
    html = driver.page_source
    bs = BeautifulSoup(html, 'html.parser')
    comments = bs.findAll('div', 'comment__StyledMargerText-sc-8s8e85-5 eYhLGU')
    #replies = bs.find('div', 'replies__Container-t86rf-0 bJiWce').findAll('div', 'comment__StyledMargerText-sc-8s8e85-5 eYhLGU')
    print(len(comments))
    for cmt in comments:
        comment.append([pro_link, cmt.text])
    #replies_txt = [reply.text for reply in replies]

    '''
    # 4. Get information about the project creaters
    #name, num_created, num_backed, num_followed = extract_creater(creater_link)
    driver.get(creater_link)
    
    # 4.1 Load the html page
    html = driver.page_source
    bs = BeautifulSoup(html, 'html.parser')
    
    # 4.2 Get information about the creater
    try:
        name = bs.find('h1', 'k-Title k-Title--quinary').text
        span = bs.findAll('span', 'k-Badge k-HorizontalNav__badge')
        num_created = int(span[0].text)
        num_backed = int(span[1].text)
        num_followed = int(int(span[2].text))
    except:
        name = 'Group creaters'
        num_created = -1
        num_backed = -1
        num_followed = -1
    
    
    creater = [
        creater_link,
        name,
        num_created,
        num_backed,
        num_followed
    ]
    
    # 7. Quit the driver
    driver.quit()
    
    return project, reward, creater

In [45]:
project, reward, creater = extract_data('https://www.kisskissbankbank.com/en/projects/backpack-guide-entrepreneuriat')
project

['https://www.kisskissbankbank.com/en/projects/backpack-guide-entrepreneuriat',
 "Backpack, le guide de l'entrepreneuriat",
 'Successful',
 '124',
 '618 pre-orders',
 'Out of 500',
 '08/08/2020',
 '557',
 'Independent media,Change your\xa0life',
 'gif',
 9,
 15.0,
 2500.0,
 4,
 27,
 564,
 'https://www.kisskissbankbank.com/en/users/maddyness',
 'fr',
 'Procurez-vous Backpack, le manuel des entrepreneur·e·s']

In [39]:
reward

[['https://www.kisskissbankbank.com/en/projects/teaserlappat',
  15.0,
  11,
  False],
 ['https://www.kisskissbankbank.com/en/projects/teaserlappat', 5.0, 7, False],
 ['https://www.kisskissbankbank.com/en/projects/teaserlappat',
  30.0,
  14,
  False],
 ['https://www.kisskissbankbank.com/en/projects/teaserlappat', 50.0, 8, False],
 ['https://www.kisskissbankbank.com/en/projects/teaserlappat', 80.0, 5, False]]

In [46]:
creater

['https://www.kisskissbankbank.com/en/users/maddyness',
 'Group creaters',
 -1,
 -1,
 -1]

In [9]:
def collect_data(start = 2, end = 3):
    
    projects = []
    rewards = []
    creaters = []
    #timelines = []
    #comments = []
    
    for i in tqdm(range(start,end)):
        
        url_i = create_url(i)
        
        # Go to the page number i
        chrome_path = './chromedriver.exe'
        driver_i = webdriver.Chrome(chrome_path)
        driver_i.get(url_i)
        time.sleep(5)

        # Accept the cookie
        cookie_question_i = driver_i.find_element_by_xpath("//*[@id=\"App-react-component\"]/div[1]/div/div/div/div/button[2]")
        cookie_question_i.click()

        # Load the html page
        html_i = driver_i.page_source
        bs_i = BeautifulSoup(html_i, 'html.parser')
        
        # Quit the driver
        driver_i.quit()
        
        # Extract data
        pro_links = bs_i.findAll('a', 'styles__StyledCrowdfundingCard-sc-1dxuhb7-0 dOcwdr k-CrowdfundingCard k-Card k-Card--light k-Card--withoutBoxShadowOnHover k-CrowdfundingCard--titlesMinHeight')
        for p in tqdm(pro_links):
            pro_link = p.get('href')
            #print(pro_link)
            project, reward, creater = extract_data(pro_link)
            projects.append(project)
            for r in reward:
                rewards.append(r)
            #print(rewards)
            creaters.append(creater)
            #for t in timeline:
                #timelines.append(t)
            #for c in comment:
                #comments.append(c)
    projects = pd.DataFrame(projects, columns = project_features)
    rewards = pd.DataFrame(rewards, columns = reward_features)
    creaters = pd.DataFrame(creaters, columns = creater_features)
    #timelines = pd.DataFrame(timelines, columns = timeline_features)
    #comments = pd.DataFrame(comments, columns = comment_features)
    return projects, rewards, creaters#, timelines, comments

### Tests

In [ ]:
create_url(3)

In [ ]:
extract_data('https://www.kisskissbankbank.com/en/projects/en-immersion-engagee')

In [21]:
projects, rewards, creaters = collect_data(2,3)
projects

100%|██████████| 9/9 [03:23<00:00, 22.58s/it]


,pro_link,title,status,percentage_fund,funded,target,end_date,backers,subcategory,thumbnail_type,number_rewards,min_price,max_price,num_news,num_comments,num_contributions,creater_link,language,short_des
0,https://www.kisskissbankbank.com/en/projects/t...,"TEASER Long-métrage ""L'APPÂT""",Successful,169,"€3,642","Out of €2,160",01/11/2021,62,"Anti racism,International solidarity,Independe...",jpg,5,5.0,80.0,0,11,64,https://www.kisskissbankbank.com/en/users/teas...,en,SOUTENEZ LE TOURNAGE DU TEASER DU LONG-MÉTRAGE...
1,https://www.kisskissbankbank.com/en/projects/e...,En immersion Engagée,Successful,682,"€13,639","Out of €2,000",01/10/2021,72,"Local Development,Indie,French manufacturing",png,7,20.0,500.0,1,14,74,https://www.kisskissbankbank.com/en/users/loic...,fr,Pré-réservez aujourd’hui votre prochaine micro...
2,https://www.kisskissbankbank.com/en/projects/r...,"Route 64, la revue francophone du jeu d'échecs...",Successful,203,406 pre-orders,Out of 200,01/10/2021,172,Independent media,jpg,7,25.0,5000.0,26,34,177,https://www.kisskissbankbank.com/en/users/fabr...,fr,"Soutenez la création de Route 64, le nouveau m..."
3,https://www.kisskissbankbank.com/en/projects/c...,"Caremitou®, maison de santé connectée qui pren...",Successful,108,27 pre-orders,Out of 25,01/10/2021,16,"Animal Welfare,Recycling,French manufacturing",gif,8,5.0,710.0,1,2,19,https://www.kisskissbankbank.com/en/users/phil...,fr,"Un bac à litière connecté, esthétique et perfo..."
4,https://www.kisskissbankbank.com/en/projects/n...,"NOMANK, les super-tisanes bio 100% françaises !",Successful,110,549 pre-orders,Out of 500,01/10/2021,122,"Organic,French manufacturing",gif,6,18.0,70.0,4,16,129,https://www.kisskissbankbank.com/en/users/mano...,fr,Soutenez-nous et remettons ensemble au goût du...
5,https://www.kisskissbankbank.com/en/projects/n...,NOUVEAU LIEU A BORDEAUX - COUP D'POUCE A BACK ...,Successful,105,"€4,185","Out of €4,000",01/10/2021,53,Local Development,jpg,9,3.0,40.0,0,10,62,https://www.kisskissbankbank.com/en/users/amel...,pt,"Covid ou pas, BACK TO sera là !"
6,https://www.kisskissbankbank.com/en/projects/a...,Abri d’Urgence : aidons les femmes victimes de...,Successful,102,"€5,108","Out of €5,000",01/10/2021,46,"Feminism,Health and Handicap",jpeg,5,30.0,500.0,2,1,49,https://www.kisskissbankbank.com/en/users/luca...,fr,"Pour de nombreuses femmes, quitter le domicile..."
7,https://www.kisskissbankbank.com/en/projects/u...,Un autre regard,Successful,131,"€4,979","Out of €3,800",01/09/2021,97,"Health and Handicap,Indie,French manufacturing",jpg,9,5.0,400.0,1,7,98,https://www.kisskissbankbank.com/en/users/mick...,fi,Soutenez le projet sur KissKissBankBank !
8,https://www.kisskissbankbank.com/en/projects/r...,Rerenga: Wines for a Revolution,Successful,113,"€9,075","Out of €8,000",01/09/2021,27,"Organic,Anti racism,Education",jpeg,7,10.0,1000.0,0,4,29,https://www.kisskissbankbank.com/en/users/nath...,en,Help fund a Paris wine shop and bookstore dedi...


In [22]:
rewards

,pro_link,price,num_backers,has_media
0,https://www.kisskissbankbank.com/en/projects/t...,15.0,11,False
1,https://www.kisskissbankbank.com/en/projects/t...,5.0,7,False
2,https://www.kisskissbankbank.com/en/projects/t...,30.0,14,False
3,https://www.kisskissbankbank.com/en/projects/t...,50.0,8,False
4,https://www.kisskissbankbank.com/en/projects/t...,80.0,5,False
...,...,...,...,...
58,https://www.kisskissbankbank.com/en/projects/r...,50.0,9,False
59,https://www.kisskissbankbank.com/en/projects/r...,150.0,2,False
60,https://www.kisskissbankbank.com/en/projects/r...,250.0,3,False
61,https://www.kisskissbankbank.com/en/projects/r...,500.0,1,False


In [23]:
creaters

,creater_link,name,num_created,num_backed,num_followed
0,https://www.kisskissbankbank.com/en/users/teas...,Teaser-Appât,1,1,0
1,https://www.kisskissbankbank.com/en/users/loic...,Loïc-Sanchez,1,0,0
2,https://www.kisskissbankbank.com/en/users/fabr...,Route 64,1,0,1
3,https://www.kisskissbankbank.com/en/users/phil...,Caremitou,1,1,0
4,https://www.kisskissbankbank.com/en/users/mano...,Nomank,1,0,0
5,https://www.kisskissbankbank.com/en/users/amel...,Julien&Amélie,1,1,0
6,https://www.kisskissbankbank.com/en/users/luca...,Make.org Foundation,1,1,0
7,https://www.kisskissbankbank.com/en/users/mick...,Mickael-Mathis,1,0,0
8,https://www.kisskissbankbank.com/en/users/nath...,Nathaniel-Ratapu,1,0,0


### Main

In [11]:
# Function to collect data and save into csv file
def save_data(start, end):
    
    # Collect 10 pages for one time to avoid the exceptions
    projects, rewards, creaters = collect_data(start,end)
    projects.to_csv('./data/projects/successful_' + str(start) + '_' + str(end-1) + '.csv')
    rewards.to_csv('./data/rewards/successful_' + str(start) + '_' + str(end-1) + '.csv')
    creaters.to_csv('./data/creaters/successful_' + str(start) + '_' + str(end-1) + '.csv')

#### Nghia: from page 1 to 400

In [43]:
for i in [61]:
    save_data(i,i+10)

 10%|█         | 1/10 [04:57<44:35, 297.28s/it]


AttributeError: 'NoneType' object has no attribute 'text'

In [42]:
project_test = pd.read_csv('./data/projects/successful_51_60.csv')
project_test.head(10)

,Unnamed: 0,pro_link,title,status,percentage_fund,funded,target,end_date,backers,subcategory,thumbnail_type,number_rewards,min_price,max_price,num_news,num_comments,num_contributions,creater_link,language,short_des
0,0,https://www.kisskissbankbank.com/en/projects/u...,Un espace d’engagement citoyen à Louvain-la-Neuve,Successful,156,"€15,645","Out of €10,000",09/28/2020,180,Local Development,jpg,8,5.0,300.0,5,30,188,https://www.kisskissbankbank.com/en/users/melo...,fr,"Donnez vie à un espace dynamique, chaleureux, ..."
1,1,https://www.kisskissbankbank.com/en/projects/e...,EVE - mon premier EP ( 6 titres),Successful,104,"€3,110","Out of €3,000",09/28/2020,52,Indie,JPG,5,10.0,200.0,0,14,58,https://www.kisskissbankbank.com/en/users/evem...,fr,J'ai besoin de vous !!😁🙏🏻❤️
2,2,https://www.kisskissbankbank.com/en/projects/s...,"Sendao, un accompagnement au mieux-être",Successful,112,"€3,911","Out of €3,500",09/27/2020,48,Change your life,jpeg,11,1.0,2000.0,3,10,59,https://www.kisskissbankbank.com/en/users/yama...,fr,"""Surperformer pour atteindre ses rêves"" : \nCo..."
3,3,https://www.kisskissbankbank.com/en/projects/p...,Portraits in Jazz,Successful,138,"€5,533","Out of €4,000",09/27/2020,95,NaN,png,4,25.0,500.0,21,18,97,https://www.kisskissbankbank.com/en/users/arna...,fr,"Ces portraits de musiciens sont comme habités,..."
4,4,https://www.kisskissbankbank.com/en/projects/c...,CLIP AMHAYA BREAK OF DAWN,Successful,100,"€3,900","Out of €3,900",09/27/2020,22,Indie,jpg,11,5.0,1000.0,0,6,26,https://www.kisskissbankbank.com/en/users/amhr...,hr,Soutenez le projet sur KissKissBankBank !
5,5,https://www.kisskissbankbank.com/en/projects/m...,"MYLÈNE L. CHAMBLAIN / EP ""Body & Soul"" + 1er V...",Successful,102,"€3,975","Out of €3,900",09/27/2020,41,Indie,png,8,5.0,500.0,2,10,49,https://www.kisskissbankbank.com/en/users/myle...,fi,Soutenez le projet sur KissKissBankBank !
6,6,https://www.kisskissbankbank.com/en/projects/l...,"L'Atelier des Sor'bières, microbrasserie",Successful,112,"€4,470","Out of €4,000",09/26/2020,37,"Local Development,French manufacturing",jpg,7,5.0,500.0,0,6,39,https://www.kisskissbankbank.com/en/users/mimo...,de,DES BIERES-BONS* OU UN SORT !\n
7,7,https://www.kisskissbankbank.com/en/projects/l...,"""Losing Touch"" - Book sale for Beirut",Successful,230,69 pre-orders,Out of 30,09/26/2020,65,International solidarity,jpg,2,25.0,50.0,0,9,66,https://www.kisskissbankbank.com/en/users/mari...,en,A book about the vanishing traditional workers...
8,8,https://www.kisskissbankbank.com/en/projects/o...,"Onetopi, un nouveau écosystème sumain",Successful,163,"€8,165","Out of €5,000",09/25/2020,160,Change your life,gif,8,10.0,300.0,3,22,163,https://www.kisskissbankbank.com/en/users/onet...,fr,Soutenez notre projet pour créer notre 1ère pl...
9,9,https://www.kisskissbankbank.com/en/projects/b...,Bachar Mar-Khalifé On / Off,Successful,204,204 pre-orders,Out of 100,09/25/2020,110,"International solidarity,Indie",jpg,20,5.0,215.0,0,18,110,https://www.kisskissbankbank.com/en/users/etie...,en,Bachar's online boutique / Pre-orders for new ...


In [32]:
project_test.head(10)

,Unnamed: 0,pro_link,title,status,percentage_fund,funded,target,end_date,backers,subcategory,thumbnail_type,number_rewards,min_price,max_price,num_news,num_comments,num_contributions,creater_link,language,short_des
0,0,https://www.kisskissbankbank.com/en/projects/t...,"Cassava, table & boulangerie artisanale engagée",Successful,106,"€4,225","Out of €4,000",01/12/2021,42,"Organic,Locavore,Local Development",png,8,10.0,400.0,1,16,46,https://www.kisskissbankbank.com/en/users/vale...,fr,Participez à la naissance de Cassava à la Réun...
1,1,https://www.kisskissbankbank.com/en/projects/i...,IBIZA HORSE VALLEY,Successful,104,"€20,880","Out of €20,000",01/12/2021,175,Animal Welfare,png,4,10.0,500.0,6,71,190,https://www.kisskissbankbank.com/en/users/jean...,fr,"Sauvons ensemble ""Ibiza Horse Valley"", le sanc..."
2,2,https://www.kisskissbankbank.com/en/projects/ob2,ENREGISTREMENT DE L'ALBUM OB2 GREGORY OTT ET V...,Successful,108,"€10,750","Out of €10,000",01/12/2021,211,Indie,jpg,7,10.0,500.0,0,53,222,https://www.kisskissbankbank.com/en/users/vinc...,hr,Soutenez le projet sur KissKissBankBank !
3,3,https://www.kisskissbankbank.com/en/projects/p...,Playschool l'album : LA GARDERIE,Successful,120,"€3,595","Out of €3,000",01/12/2021,95,Indie,jpg,7,10.0,1000.0,4,22,98,https://www.kisskissbankbank.com/en/users/emma...,fr,"Collectif d’artistes, Playschool a besoin de v..."
4,4,https://www.kisskissbankbank.com/en/projects/s...,S!CK soutient l'indépendance avec le double-nu...,Successful,876,876 pre-orders,Out of 100,01/12/2021,312,"French manufacturing,Independent media",gif,12,14.0,225.0,10,41,325,https://www.kisskissbankbank.com/en/users/yoan...,fr,"164 pages, 2 couvertures et 0 publicité !"
5,5,https://www.kisskissbankbank.com/en/projects/t...,"TEASER Long-métrage ""L'APPÂT""",Successful,169,"€3,642","Out of €2,160",01/11/2021,62,"Anti racism,International solidarity,Independe...",jpg,5,5.0,80.0,0,11,64,https://www.kisskissbankbank.com/en/users/teas...,en,SOUTENEZ LE TOURNAGE DU TEASER DU LONG-MÉTRAGE...
6,6,https://www.kisskissbankbank.com/en/projects/e...,En immersion Engagée,Successful,682,"€13,639","Out of €2,000",01/10/2021,72,"Local Development,Indie,French manufacturing",png,7,20.0,500.0,1,14,74,https://www.kisskissbankbank.com/en/users/loic...,fr,Pré-réservez aujourd’hui votre prochaine micro...
7,7,https://www.kisskissbankbank.com/en/projects/r...,"Route 64, la revue francophone du jeu d'échecs...",Successful,203,406 pre-orders,Out of 200,01/10/2021,172,Independent media,jpg,7,25.0,5000.0,26,34,177,https://www.kisskissbankbank.com/en/users/fabr...,fr,"Soutenez la création de Route 64, le nouveau m..."
8,8,https://www.kisskissbankbank.com/en/projects/c...,"Caremitou®, maison de santé connectée qui pren...",Successful,108,27 pre-orders,Out of 25,01/10/2021,16,"Animal Welfare,Recycling,French manufacturing",gif,8,5.0,710.0,1,2,19,https://www.kisskissbankbank.com/en/users/phil...,fr,"Un bac à litière connecté, esthétique et perfo..."
9,9,https://www.kisskissbankbank.com/en/projects/n...,"NOMANK, les super-tisanes bio 100% françaises !",Successful,110,549 pre-orders,Out of 500,01/10/2021,122,"Organic,French manufacturing",gif,6,18.0,70.0,4,16,129,https://www.kisskissbankbank.com/en/users/mano...,fr,Soutenez-nous et remettons ensemble au goût du...


#### Anna: from page 401 to 800

#### Madellein: from page 801 to 1200

#### Yukun: from page 1201 to 1600